## Completions

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/completions)


In [ ]:
# Completion
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."

response = openai.Completion.create(
    model="llamacpp",
    prompt=prompt,
    temperature=1.31,
    max_tokens=8192,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    stream=False,
)
message = response.choices[0].text.strip()
print(message)


## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [ ]:
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."
messages = [{"role": "system", "content": prompt}]

response = openai.ChatCompletion.create(
    model="Local-LLM",
    messages=messages,
    temperature=1.31,
    max_tokens=8192,
    top_p=1.0,
    n=1,
    stream=False,
)
message = response.choices[0].message.content.strip()
print(message)

## Embeddings

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/embeddings)

The embeddings endpoint it currently uses is an ONNX embedder with 256 max tokens.


In [ ]:
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."

response = openai.Embedding.create(
    input=prompt,
    engine="Local-LLM",
)
embedding = response.embedding
print(embedding)
